<a href="https://colab.research.google.com/github/kchaitanya954/Machine-learning-technologies/blob/main/ML_Engineer_Mini_Project_Problem_A_Krishna_Chaitanya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
#!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
#!pip install fastai
import fastai
from fastai import *
from fastai.text import * 
from functools import partial
import io
import os
import numpy as np
import pandas as pd


In [3]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks/JEOPARDY_CSV.csv',na_values=['None'])
print(df.shape)
df.head()

(216930, 7)


,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [4]:
df.isnull().sum()

Show Number       0
 Air Date         0
 Round            0
 Category         0
 Value         3634
 Question         0
 Answer           3
dtype: int64

In [5]:
df = df.dropna()
df = df.reset_index(drop=True)

In [6]:
df['ValueNum'] = df[' Value'].apply(lambda value: int(value.replace(',', '').replace('$', '')))

In [7]:
def binning(value):
    if value < 1000:
        return np.round(value, -2)
    elif value < 10000:
        return np.round(value, -3)
    else:
        return np.round(value, -4)

In [8]:
df['ValueBins'] = df['ValueNum'].apply(binning)

In [9]:
df[' Question'] = df[' Question'].str.replace("[^a-zA-Z]", " ")

In [10]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
# tokenization 
tokenized_doc = df[' Question'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])



In [13]:
# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df[' Question'] = detokenized_doc

In [14]:
df = df[[' Question', 'ValueBins']]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213293 entries, 0 to 213292
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0    Question  213293 non-null  object
 1   ValueBins  213293 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.3+ MB


In [16]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['ValueBins'], test_size = 0.2, random_state = 12)

In [17]:
print(df_trn.shape, df_val.shape)

(170634, 2) (42659, 2)


In [18]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


ULMFiT Transfer learning model is used.
This method involves fine-tuning a pre-trained language model (LM), trained on the Wikitext 103 dataset, to a new dataset in such a manner that it does not forget what it previously learned.

In [20]:
learn = language_model_learner(data_lm,  arch = AWD_LSTM, pretrained = True, drop_mult=0.7)

In [24]:
learn.fit_one_cycle(1, 1e-1)

epoch,train_loss,valid_loss,accuracy,time
0,1.412118,1.066365,0.598583,00:12


In [ ]:
learn.summary()

The accuracy of the model is 60%.

In [24]:
!nvidia-smi


Thu Mar 18 17:20:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W /  70W |   1690MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
import tensorflow as tf
print(tf.version.VERSION)

2.4.1


In [28]:
!git clone --depth 1 -b v2.4.0 https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 752, done.
remote: Counting objects: 100% (752/752), done.
remote: Compressing objects: 100% (651/651), done.
remote: Total 752 (delta 166), reused 371 (delta 91), pack-reused 0
Receiving objects: 100% (752/752), 1.22 MiB | 12.58 MiB/s, done.
Resolving deltas: 100% (166/166), done.
Note: checking out '47e65c1aedf4181faaf5535f06c26dbcb083c4ad'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [31]:
# install requirements to use tensorflow/models repository
!pip install -Uqr models/official/requirements.txt
# you may have to restart the runtime afterwards

     |████████████████████████████████| 6.5MB 9.7MB/s 
     |████████████████████████████████| 225kB 52.2MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 15.3MB 154kB/s 
     |████████████████████████████████| 9.9MB 34.9MB/s 
     |████████████████████████████████| 296kB 55.6MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 27.4MB 103kB/s 
     |████████████████████████████████| 174kB 56.2MB/s 
     |████████████████████████████████| 3.7MB 50.5MB/s 
     |████████████████████████████████| 706kB 49.5MB/s 
     |████████████████████████████████| 358kB 58.2MB/s 
     |████████████████████████████████| 11.6MB 50.2MB/s 
     |████████████████████████████████| 645kB 51.8MB/s 
     |████████████████████████████████| 37.6MB 183kB/s 
     |████████████████████████████████| 2.2MB 43.4MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 1.2MB 44.2MB

In [ ]:
!pip install tf-models-official

In [35]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import sys
sys.path.append('models')
from official.nlp.data import classifier_data_lib
from official.nlp.bert import tokenization
from official.nlp import optimization

In [36]:
train_df, valid_df = train_test_split(df, random_state=1, train_size = 0.8, stratify = df.ValueBins.values)
train_df.shape, valid_df.shape

((170634, 2), (42659, 2))

In [37]:
with tf.device('/cpu:0'):
  train_data = tf.data.Dataset.from_tensor_slices((train_df[' Question'].values, train_df['ValueBins'].values))
  valid_data = tf.data.Dataset.from_tensor_slices((valid_df[' Question'].values, valid_df['ValueBins'].values))

  for text, label in train_data.take(1):
    print(text)

tf.Tensor(b'VIDEO DAILY DOUBLE Hi I Chuck Woolery Going date people pay known type treat', shape=(), dtype=string)


In [38]:
label_list= sorted(df.ValueBins.unique())
 # maximum length of (token) input sequences
max_seq_length = 50
train_batch_size =32

# Get BERT layer and tokenizer:
# More details here: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [39]:
def to_feature(text, label, label_list=label_list, max_seq_length=max_seq_length, tokenizer=tokenizer):
  example = classifier_data_lib.InputExample(guid = None, text_a=text.numpy(), text_b=None, label=label.numpy())

  feature = classifier_data_lib.convert_single_example(0, example, label_list, max_seq_length, tokenizer)

  return (feature.input_ids, feature.input_mask, feature.segment_ids, feature.label_id)


In [40]:
def to_feature_map(text, label):
  
  input_ids, input_mask, segment_ids, label_id = tf.py_function(to_feature, inp= [text, label], Tout=[tf.int32, tf.int32, tf.int32, tf.int32])

  input_ids.set_shape([max_seq_length])
  input_mask.set_shape([max_seq_length])
  segment_ids.set_shape([max_seq_length])
  label_id.set_shape([])

  x={
      'input_word_ids':input_ids,
     'input_mask': input_mask,
     'input_type_ids': segment_ids
  }
  return (x, label_id)

In [41]:
with tf.device('/cpu:0'):
  # train
  train_data= (train_data.map(to_feature_map, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  .shuffle(1000)
  .batch(32, drop_remainder=True)
  .prefetch(tf.data.experimental.AUTOTUNE)
  )

  # valid
  valid_data= (valid_data.map(to_feature_map, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  .batch(32, drop_remainder=True)
  .prefetch(tf.data.experimental.AUTOTUNE)
  )

In [42]:
# Building the model
def create_model():
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
  input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="input_type_ids")
  pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])

  drop = tf.keras.layers.Dropout(0.4)(pooled_output)

  output = tf.keras.layers.Dense(21, activation='softmax', name="output")(drop)

  model = tf.keras.Model(inputs={'input_word_ids': input_word_ids, 'input_mask': input_mask, 'input_type_ids':input_type_ids},
                         outputs = output)

  return model

In [44]:
model= create_model()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2),
                    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
                    metrics= ['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [45]:
history = model.fit(train_data, validation_data=valid_data, epochs=2, verbose=1)

Epoch 1/2
5332/5332 [==============================] - 1890s 352ms/step - loss: 3.4005 - accuracy: 0.1406 - val_loss: 2.6624 - val_accuracy: 0.1494
Epoch 2/2
5332/5332 [==============================] - 1873s 351ms/step - loss: 3.3344 - accuracy: 0.1427 - val_loss: 3.1987 - val_accuracy: 0.1494


In [47]:
sample_texts =['who is worlds best cricketer']
test_data= tf.data.Dataset.from_tensor_slices((sample_texts, [0]*(len(sample_texts))))
test_data= (test_data.map(to_feature_map).batch(1))
preds= model.predict(test_data)

np.argmax(preds,axis=1)

array([8])

ULMFiT Transfer learning model gives better accuracy for the data